In [1]:
%matplotlib widget

In [2]:
import xarray as xr
import geoviews as gv
import rasterio
from planetarypy import geotools as gt
import pvl
from heat1d.data import L3DivData

In [3]:
root = Path("/luna4/maye/l3_data/")

In [4]:
labels = sorted(list(root.glob("*.lbl")))
images = sorted(list(root.glob("*.jp2")))

In [5]:
class STData:
    root = Path("/luna4/maye/l3_data")
    
    def __init__(self, cycle):
        self.cycle = cycle
        self.l3divdata = L3DivData(cycle=cycle)
        self.img = gt.ImgData(str(root / self.l3divdata.fname))

    def read_window(self, ul_lon=0, ul_lat=1, width_degrees=1):
        ul = gt.Point.copy_geodata(self.img.center, lon=ul_lon, lat=ul_lat)
        ul.lonlat_to_pixel()
        lr = gt.Point.copy_geodata(ul, lon=ul_lon+width_degrees, lat=ul_lat-width_degrees)
        lr.lonlat_to_pixel()
        win = gt.Window(ulPoint=ul, lrPoint=lr)
        self.img.read_window(win)
        
        
    @property
    def label(self):
        return pvl.load(root / self.l3divdata.label)
    
    @property
    def fname(self):
        return root / self.l3divdata.fname
    
    @property
    def SCALING_FACTOR(self):
        return self.label['UNCOMPRESSED_FILE']["IMAGE"]["SCALING_FACTOR"]
        
    @property
    def OFFSET(self):
        return self.label['UNCOMPRESSED_FILE']["IMAGE"]["OFFSET"]
        
    @property
    def NODATA(self):
        return self.label['UNCOMPRESSED_FILE']["IMAGE"]["MISSING_CONSTANT"]
    
    @property
    def data(self):
        data = self.img.data.astype('float')
        data[data == self.NODATA] = np.nan
        return data
    
    @property
    def scaled_data(self):
        return self.data*self.SCALING_FACTOR + self.OFFSET
    
    def get_pixel(self, xoff, yoff):
        value = np.squeeze(self.img.ds.ReadAsArray(xoff, yoff, 1, 1))
        if value == self.NODATA:
            return np.nan
        else:
            return value*self.SCALING_FACTOR + self.OFFSET

    def plot_window(self):
        plt.figure()
        plt.imshow(self.scaled_data, cmap='plasma')
        plt.colorbar()
    
    @property
    def window_mean(self):
        return np.nanmean(self.scaled_data)
    
    @property
    def window_std(self):
        return np.nanstd(self.scaled_data)
    
    @property
    def n_valid(self):
        return np.count_nonzero(~np.isnan(self.data))

In [6]:
fname = "/u/paige/maye/src/heat1d/python/heat1d/data/div_mapcycles_RA.txt"
cycles = pd.read_table(fname, header=None, squeeze=True)

In [9]:
from dask import delayed, compute

In [11]:
results = []

def get_mean_std(cycle):
    d = {}
    d['cycle'] = cycle
    stdata = STData(cycle)
    stdata.read_window(ul_lon=311-180, ul_lat=21)
    d['st_mean'] = stdata.window_mean
    d['st_std'] = stdata.window_std
    d['n_valid'] = stdata.n_valid
    d['pixel'] = stdata.get_pixel(23040, 10240)
    return d

for cycle in cycles:
    results.append(delayed(get_mean_std)(cycle))

In [12]:
results = compute(*results)

/luna4/maye/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/luna4/maye/miniconda3/envs/py37/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [13]:
temperatures = pd.DataFrame(results)

In [14]:
from diviner.pipetools import read_pipe_file

from sh import divgdr, pcons

def get_avg_localtime(cycle, lat_start, lon_start, dlat=1, dlon=1):
    d = {}
    d['cycle'] = cycle
    fname = f"ltim_avg_{cycle}.pipe"
    print(fname)
    if not Path(fname).exists():
        divgdr(
            "res=128",
            f"date={cycle}",
            "dn=N",
            "extract=ltim_avg",
            f"lat={lat_start},{lat_start+dlat}",
            f"lon={lon_start},{lon_start+dlon}",
            _out=fname,
        )
    df = read_pipe_file(fname)
    df[df<-9998] = np.nan
    df = df.dropna(how='any')
    d['lt_mean'] = df.ltim_avg.mean()
    d['lt_std'] = df.ltim_avg.std()
    return d

In [15]:
results = []

for cycle in cycles:
    results.append(delayed(get_avg_localtime)(cycle, 20, 311-180))

In [16]:
results = compute(*results)

ltim_avg_20140812.pipe
ltim_avg_20121123.pipe
ltim_avg_20120929.pipe
ltim_avg_20120709.pipe
ltim_avg_20160527.pipe
ltim_avg_20100303.pipe
ltim_avg_20141101.pipe
ltim_avg_20130422.pipe
ltim_avg_20160221.pipe
ltim_avg_20151022.pipe
ltim_avg_20150801.pipe
ltim_avg_20141005.pipe
ltim_avg_20151113.pipe
ltim_avg_20121026.pipe
ltim_avg_20110625.pipe
ltim_avg_20130519.pipe
ltim_avg_20120618.pipe
ltim_avg_20090906.pipe
ltim_avg_20150925.pipe
ltim_avg_20150828.pipe
ltim_avg_20150428.pipe
ltim_avg_20100107.pipe
ltim_avg_20140315.pipe
ltim_avg_20111107.pipe
ltim_avg_20111222.pipe
ltim_avg_20120210.pipe
ltim_avg_20130726.pipeltim_avg_20130822.pipe
ltim_avg_20130102.pipe
ltim_avg_20100827.pipe
ltim_avg_20130226.pipe

ltim_avg_20100330.pipeltim_avg_20140216.pipe

ltim_avg_20100731.pipeltim_avg_20131016.pipe

ltim_avg_20160416.pipeltim_avg_20110320.pipe

ltim_avg_20100620.pipe
ltim_avg_20140119.pipe
ltim_avg_20111010.pipe
ltim_avg_20160319.pipe
ltim_avg_20091003.pipe
ltim_avg_20140411.pipe
ltim_avg_20

In [17]:
len(results)

104

In [18]:
ltimes = pd.DataFrame(results)
ltimes.head()

,cycle,lt_mean,lt_std
0,20090705,NaN,NaN
1,20090713,4.390132,0.039783
2,20090810,2.614682,0.039575
3,20090906,0.836064,0.000000
4,20091003,23.047278,0.041579


In [19]:
temps = temperatures.merge(ltimes, on='cycle')
temps.head()

,cycle,st_mean,st_std,n_valid,pixel,lt_mean,lt_std
0,20090705,NaN,NaN,0,NaN,NaN,NaN
1,20090713,93.044144,1.578672,4464,NaN,4.390132,0.039783
2,20090810,93.962200,1.072995,4300,NaN,2.614682,0.039575
3,20090906,NaN,NaN,0,NaN,0.836064,0.000000
4,20091003,98.122448,2.220140,2386,NaN,23.047278,0.041579


In [20]:
temps['time_since_noon'] = temps.lt_mean - 12
temps.loc[temps.time_since_noon<0, "time_since_noon"] = temps.time_since_noon + 24

In [21]:
temps.head()

,cycle,st_mean,st_std,n_valid,pixel,lt_mean,lt_std,time_since_noon
0,20090705,NaN,NaN,0,NaN,NaN,NaN,NaN
1,20090713,93.044144,1.578672,4464,NaN,4.390132,0.039783,16.390132
2,20090810,93.962200,1.072995,4300,NaN,2.614682,0.039575,14.614682
3,20090906,NaN,NaN,0,NaN,0.836064,0.000000,12.836064
4,20091003,98.122448,2.220140,2386,NaN,23.047278,0.041579,11.047278


In [22]:
temps.dropna(how='any', inplace=True, subset=['lt_mean', 'lt_std'])

In [23]:
temps['year'] = np.floor(temps.cycle/10000).astype('int')

In [24]:
import seaborn as sns

In [25]:
sns.set_theme()

In [26]:
g = sns.relplot(x="time_since_noon", y='pixel', data=temps, height=5, aspect=1)
g.ax.set_title("Pixel 0,0 ST temps")
g.ax.set_ylabel("Surface Temp [K]")
g.ax.set_ylim((88, 116))
g.ax.set_xlim((6, 20))
g.fig.set_size_inches(8,5)
g.fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
temps.pixel.notnull().sum()

12

In [28]:
grid = sns.relplot(
    x="time_since_noon",
    y="st_mean",
    size="st_std",
    hue="n_valid",
    data=temps,
    palette="viridis_r",
)
grid.ax.set_title("Surface temps at lat/lon 0/0, box of 1 degree")
grid.ax.set_ylim((88, 116))
grid.ax.set_xlim((6, 20))
grid.fig.set_size_inches(8, 5)
grid.fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
get_avg_localtime(failed[0], 0,0)

ltim_avg_20090713.pipe


{'cycle': 20090713,
 'lt_mean': 4.390131663288417,
 'lt_std': 0.039783077654775394}

In [53]:
img = gt.ImgData(fname)

In [54]:
img.projection

'GEOGCS["GCS_Moon_2000",DATUM["Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

In [55]:
p = img.center

In [56]:
p.pixel_to_lonlat()

(0.0, 0.0)

In [57]:
p

Pixel: (23040, 10240)
Map: (0.0, 0.0)
Lon/Lat: (0.0, 0.0)

In [58]:
gt.Point.copy_geodata(p, sample=0, line=0)

Pixel: (0, 0)
Map: (-180.0, 80.0)
Lon/Lat: (180.0, 80.0)

In [59]:
img.read_center_window()

In [60]:
# def get_degree_window(img, start_lon=0, start_lat=0, degrees=1):
width_degrees = 1
ul_lon = 0
ul_lat = 1
ul = gt.Point.copy_geodata(img.center, lon=ul_lon, lat=ul_lat)
ul.lonlat_to_pixel()
lr = gt.Point.copy_geodata(ul, lon=ul_lon+width_degrees, lat=ul_lat-width_degrees)
lr.lonlat_to_pixel()
win = gt.Window(ulPoint=ul, lrPoint=lr)
win

UL:
Pixel: (23040.0, 10112.0)
Map: (0.0, 1.0)
Lon/Lat: (0, 1)
LR:
Pixel: (23168.0, 10240.0)
Map: (1.0, 0.0)
Lon/Lat: (1, 0)
Width: None
Center:
None

In [61]:
win.get_gdal_window()

[23040, 10112, 128, 128]

In [65]:
img.read_window(win)
data = img.data

In [66]:
data = data.astype('float')

In [67]:
data[data < 0] = np.nan

In [69]:
data = convert_DN(data)

In [70]:
np.nanmean(data)

111.19537225495448

In [73]:
np.nanstd(data)

1.352176847229142

In [71]:
data.shape

(128, 128)

In [74]:
%matplotlib widget

In [76]:
plt.figure()
plt.imshow(data)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [216]:
img.data[img.data==-32768]= 0

In [217]:
img.data.mean()

0.0

In [176]:
img.read_center_window()

In [177]:
from rasterio.windows import Window

In [178]:
ds.read(1, window=Window(23040, 10111, 128, 128))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [179]:
img.center

Pixel: (23040, 10240)
Map: (0.0, 0.0)
Lon/Lat: (0.0, 0.0)

In [82]:
da = xr.open_rasterio(fname, chunks={'band':1, 'x':2048, 'y':1024})

In [83]:
da

,Array,Chunk
Bytes,1.89 GB,4.19 MB
Shape,"(1, 20480, 46080)","(1, 1024, 2048)"
Count,461 Tasks,460 Chunks
Type,int16,numpy.ndarray


In [89]:
da.isel(x=slice(23040,23160), y=slice(10200, 103128)).max().compute()

<xarray.DataArray ()>
array(7509, dtype=int16)

In [ ]:
da.plot

In [112]:
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

In [113]:
import holoviews as hv

In [117]:
hvds = hv.Dataset(da)

DataError: xarray DataArray does not define a name and Dataset does not define a default value dimension. Give the DataArray a name or supply an explicit vdim.

XArrayInterface expects gridded data, for more information on supported datatypes see http://holoviews.org/user_guide/Gridded_Datasets.html

In [11]:
from rasterio.warp import transform

In [12]:
ny, nx = len(da['y']), len(da['x'])
x, y = np.meshgrid(da['x'], da['y'])

In [13]:
lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
                     x.flatten(), y.flatten())

CPLE_NotSupportedError: Cannot find coordinate operations from 'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["unknown",1737400,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Reference meridian",0,ANGLEUNIT["degree",0.0174532925199433,ID["EPSG",9122]]]],CONVERSION["unnamed",METHOD["Equidistant Cylindrical",ID["EPSG",1028]],PARAMETER["Latitude of 1st standard parallel",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]' to 'EPSG:4326'

# datashader

https://aetperf.github.io/2018/09/19/Nighttime-Lights-with-Rasterio-and-Datashader.html

In [218]:
import rasterio
from rasterio.mask import mask
import os
import datashader as ds
from datashader import transfer_functions as tf
import xarray as xr
from colorcet import palette
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import json

In [219]:
raster = rasterio.open(fname)

In [220]:
raster.crs

CRS.from_wkt('GEOGCS["GCS_Moon_2000",DATUM["Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')

In [221]:
raster.bounds

BoundingBox(left=-180.0, bottom=-80.0, right=180.0, top=80.0)

In [222]:
da = xr.open_rasterio(fname)

In [223]:
cmap=palette['fire']
bg_col = 'black'

In [ ]:
cvs = ds.Canvas(plot_width=800, plot_height=800)
img = tf.shade(cvs.raster(da), cmap=cmap)
img = tf.set_background(img, bg_col)
img